In [1]:
import spacy
from spacy.en import English

In [170]:
!dir
!pip install datetime
import datetime

 Volume in drive C is WINDOWS
 Volume Serial Number is AA1C-00BD

 Directory of C:\Users\Zane_Harris\Documents\GitHub\ga-students-datascience\lessons\lesson-14\code\starter-code

04/14/2016  08:30 PM    <DIR>          .
04/14/2016  08:30 PM    <DIR>          ..
04/14/2016  06:42 PM    <DIR>          .ipynb_checkpoints
02/25/2016  07:08 PM               549 capture-tweets.py
04/05/2016  07:21 PM             4,151 helper.ipynb
02/25/2016  07:08 PM               864 starter-code-14.py
04/05/2016  07:21 PM             6,086 starter-code-final.ipynb
04/06/2016  11:36 AM            14,804 starter-code-final_retry.ipynb
04/14/2016  08:30 PM            21,511 structuring_nlp_helper.ipynb
02/25/2016  07:08 PM               979 twitter.py
04/05/2016  11:55 PM             9,640 Untitled.ipynb
               8 File(s)         58,584 bytes
               3 Dir(s)  130,490,863,616 bytes free


In [175]:
def structure_file(string):
    string_split = string.split("|") # list of length 4
    
    first_name = string_split[0]
    last_name  = string_split[1]
    
    age = int(string_split[2])
    
    emails = string_split[3].split(",")
    email_1 = emails[0]
    email_2 = emails[1]
    emails = [email_1,email_2]
    
    dob = get_dob_from_string(string_split[4])
    
    final_obj = { 
        "first_name" : first_name,
        "last_name"  : last_name,
        "age" : age, 
        "emails" : emails,
        "dob" : dob
    }
    
    return final_obj

def get_dob_from_string(dob):
    return datetime.datetime.strptime(dob, "%Y-%m-%d").date()
def get_month_from_string(dob):
    return datetime.datetime.strptime(dob, "%Y-%m-%d").month
def bin_month_on_season(month):
    if month in [1,2,3]:
        return 1
    elif month in [4,5,6]:
        return 2
    elif month in [7,8,9]:
        return 3
    else:
        return 4

In [176]:
sentence_1 = "Bilind|Hajer|27|bilind07@gmail.com,bhajer3@gatech.edu|1989-05-28"
sentence_2 = "John|Smith|24|JS12@gmail.com,jSmith3@gatech.edu|1990-04-20"
sentence_3 = "Sarah|Adams|22|sarah22@gmail.com,sAdams3@gatech.edu|1991-03-30"
sentence_4 = "Rob|Adams|24|rob24@gmail.com,rAdams3@gatech.edu|1990-02-15"
textfile = [sentence_1,sentence_2,sentence_3,sentence_4]

In [177]:
textfile_structured = map(structure_file,textfile)

In [178]:
for obj in textfile_structured:
    print(obj)

{'dob': datetime.date(1989, 5, 28), 'first_name': 'Bilind', 'last_name': 'Hajer', 'age': 27, 'emails': ['bilind07@gmail.com', 'bhajer3@gatech.edu']}
{'dob': datetime.date(1990, 4, 20), 'first_name': 'John', 'last_name': 'Smith', 'age': 24, 'emails': ['JS12@gmail.com', 'jSmith3@gatech.edu']}
{'dob': datetime.date(1991, 3, 30), 'first_name': 'Sarah', 'last_name': 'Adams', 'age': 22, 'emails': ['sarah22@gmail.com', 'sAdams3@gatech.edu']}
{'dob': datetime.date(1990, 2, 15), 'first_name': 'Rob', 'last_name': 'Adams', 'age': 24, 'emails': ['rob24@gmail.com', 'rAdams3@gatech.edu']}


In [180]:
# give me only student's bilind's record
student = filter(
    lambda obj: obj["first_name"].lower() == "bilind",
    textfile_structured
)

young_students = filter(lambda obj: obj["dob"] >= datetime.date(1991, 1, 1), textfile_structured)

In [181]:
print(student[0]["first_name"])

for record in student:
    print record["first_name"]
    
student_24_over = filter(lambda obj: obj["age"] >= 24, textfile_structured)

"adams" in "rAdams3@gatech.edu".lower()

def check_name_adams(obj):
    return "adams" in obj["emails"][0].lower() or "adams" in obj["emails"][1].lower()

Bilind
Bilind


In [182]:
print(student)

[{'dob': datetime.date(1989, 5, 28), 'first_name': 'Bilind', 'last_name': 'Hajer', 'age': 27, 'emails': ['bilind07@gmail.com', 'bhajer3@gatech.edu']}]


In [183]:
# get all students atleast 24 years old

In [184]:
new_sentence = "Hi, my name is Bilind, I like to run, jog, and eat"

In [185]:
sentence_to_unicode = unicode(new_sentence, errors='ignore')

In [186]:
print(sentence_to_unicode)

Hi, my name is Bilind, I like to run, jog, and eat


In [187]:
nlp_toolkit = English()

In [188]:
parsed = nlp_toolkit(sentence_to_unicode) # parse or tokenize the unicode object

In [189]:
entities = parsed.ents # grade entities of tokenized unicode object

In [195]:
def mentions_name(entities, name):
    for entity in entities:
        if entity.text == name:
            return True
    return False

In [196]:
def is_an_action(token):
    if(token.pos == spacy.parts_of_speech.VERB):
        return True
    return False

In [197]:
def get_actions(parsed):
    actions = filter(is_an_action,parsed)
    actions = map(lambda token: token.lemma_ , actions)
    return actions

In [198]:
print mentions_name(entities,"Bilind")

True


In [62]:
print get_actions(parsed)

[u'be', u'like', u'run', u'eat']


In [63]:
# Write a function that can take a take a sentence parsed by `spacy` and 
# identify if it mentions a company named 'Google'. 
# Remember, `spacy` can find entities and code them `ORG` if they are a company.

In [64]:
# Write a function that can take a sentence parsed by `spacy` 
# and return the verbs of the sentence (preferably lemmatized)

In [65]:
# Write a function that identifies countries - HINT: the entity label for 
# countries is GPE (or GeoPolitical Entity)

In [194]:
from gensim.models.word2vec import Word2Vec
if __name__ == '__main__':
    # Loading the tweet data
    tweets = [unicode(tweet, errors='ignore') for tweet in open('../../assets/dataset/captured-tweets.txt', 'r')]

    # Setting up spacy
    nlp_toolkit = English()

    # For each tweet, parse it using `spacy` and print out if the tweet 
    # has 'release' or 'announce' as a verb.
    for tweet in tweets:
        parsed = nlp_toolkit(tweet)

        if mentions_company(parsed, 'Google'):
            actions = get_actions(parsed)
            if 'release' in actions or 'announce' in actions:
                print(tweet)

    # Solution to 1f
    # Re-run (d) find country tweets that discuss 'Iran' announcing or releasing.
    for tweet in tweets:
        parsed = nlp_toolkit(tweet)

        if mentions_country(parsed, 'Iran'):
            actions = get_actions(parsed)
            if 'release' in actions or 'announce' in actions:
                print(tweet)

    # First take the collection of tweets and tokenize them using `spacy`

    # I decided to lemmatized the verbs for easier searching and keep symbols
    # and punctuations

    text_split = [[x.text if x.pos != spacy.parts_of_speech.VERB else x.lemma_ 
                    for x in nlp_toolkit(t)] for t in tweets]

    model = Word2Vec(text_split, size=100, window=4, min_count=5, workers=4)

    model.most_similar(positive=['Syria'])

    # Filter tweets down to those that mention 'Iran' or similar entities and 
    # 'war' or similar entities

    # a: Using spacy
    for tweet in tweets:
        parsed = nlp_toolkit(tweet)
        if mentions_country(parsed, 'Iran') or mentions_country(parsed, 'Iraq'): # ... you could add more
            if 'attack' in get_actions(parsed):
                print(tweet)

    # b: using similarity scores
    for tweet in tweets:
        parsed = nlp_toolkit(tweet)

        similarity_to_iran = max([model.similarity('Iran', tok.text) for tok in parsed if tok.text in model.vocab], 0)
        similarity_to_war = max([model.similarity('war', tok.text) for tok in parsed if tok.text in model.vocab], 0)
        if similarity_to_iran > 0.75 and similarity_to_war > 0.75:
            print(similarity_to_iran, similarity_to_war, tweet)


ImportError: cannot import name interfaces